In [3]:
import pandas as pd
import regex as re

In [44]:
data = pd.read_csv("/mtsamples.csv", index_col=[0])
data.head()

,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4999 entries, 0 to 4998
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        4999 non-null   object
 1   medical_specialty  4999 non-null   object
 2   sample_name        4999 non-null   object
 3   transcription      4966 non-null   object
 4   keywords           3931 non-null   object
dtypes: object(5)
memory usage: 234.3+ KB


In [46]:
# creating a new temp column to identify the columns with any numerical values
temp = []
for i in data.description:
  temp.append(re.findall(r"\b\d{1,2}", i))
# if there are no numerical values in the column, assign None
temp = [i if len(i)>0 else None for i in temp]

In [48]:
# creating temp column
data['temp'] = temp
data.head()

,description,medical_specialty,sample_name,transcription,keywords,temp
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...",[23]
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...",None
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart...",None
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",[2]
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",[2]


In [53]:
#filter the columns that are not none and create a new dataframe
data_num = data[~data.temp.isna()]
data_num.head()

,description,medical_specialty,sample_name,transcription,keywords,temp
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...",[23]
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",[2]
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",[2]
5,Morbid obesity. Laparoscopic antecolic anteg...,Bariatrics,Laparoscopic Gastric Bypass,"PREOPERATIVE DIAGNOSIS: , Morbid obesity.,POST...","bariatrics, gastric bypass, eea anastomosis, r...",[30]
7,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 3,"2-D ECHOCARDIOGRAM,Multiple views of the heart...","cardiovascular / pulmonary, 2-d echocardiogram...",[2]


#### Sample Edge cases

> The patient had ultrasound done on admission that showed gestational age of 38-2/7 weeks.<br>
>  External cephalic version.  A 39-week intrauterine pregnancy with complete breech presentation. <br>
>   She underwent a lipomyomeningocele repair at 3 days of age and then again at 3-1/2 years of age.

In [62]:
# data_num.to_csv("numerics_data.csv")

# 23-year-old, 16-month-old, 15-day-old,   6-week-old, 10-1/2-year-old, 90-years-old, 2 months old, 3-1/2 years of age, 50 years old


In [86]:
age_list = []
for i in data.description:
  # print(re.findall(r"\b(\d+-year(?:s)?(?:-old))\b|",i))
  # print(re.findall(r"\b(\d+\s(?:year(?:s)|month(?:s)|day(?:s))\sold)",i))
  # pattern to match age type  '2 (year/years) of age'
  match_1 = re.findall(r"\d+\syear(?:s)\sof\sage", i)
  if not (match_1):
    # pattern to match age type '2 (year/years)/(day/days)/(month/months) old
    match_2 = re.findall(r"\b\d+\s(?:year(?:s)?|month(?:s)?|day(?:s)?)\sold", i)
    if not match_2:
      # pattern to match age type '2(-)(year/years)/(month/months)/(day/days)(-)old
      match_3 = re.findall(r"\b\d+[-]?[/]?\w+[\s-]?\w+\b", i)
      filtered_match_3 = []
      for j in match_3:
        # as the above pattern matches words that are not age, filtering for age based on the obtained results
        if ('year' in j) or ('old' in j) or ('day' in j) or ('month' in j) or ('day' in j):
          if 'well' not in j and 'recheck' not in j and 'revocation' not in j and 'history' not in j:
            filtered_match_3.append(j)
      age_list.append(filtered_match_3)
    else:
      age_list.append(match_2)
  else:
    age_list.append(match_1)



In [90]:
# appending the extracted ages to the new age column
age = [i if len(i)>0 else None for i in age_list]
data['age']= age

In [92]:
data.head()

,description,medical_specialty,sample_name,transcription,keywords,transc_descrp,age
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...",A 23-year-old white female presents with comp...,[23-year-old]
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...",Consult for laparoscopic gastric bypass.PAST ...,None
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart...",Consult for laparoscopic gastric bypass.HISTO...,None
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...","2-D M-Mode. Doppler. 2-D M-MODE: , ,1. Left...",None
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",2-D Echocardiogram1. The left ventricular ca...,None


In [33]:
# data[data.age.isna()].description.to_csv('Noage.csv')

In [7]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
result = pipe("")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

In [45]:
data.transcription.fillna(value=' ', inplace=True)
data['transc_descrp'] = data.description+data.transcription

In [ ]:
age = []
medic = []

for i in data.transc_descrp:
  age_lst = []
  medic_lst = []
  # print(i)
  ner_text = pipe(i)
  print(ner_text)
  for i in ner_text:
    if i['entity_group'].lower() == 'age':
      age_lst.append(i['word'])
    elif i['entity_group'].lower() == 'medication':
      medic_lst.append(i['word'])
    else:
      continue
  if age_lst:
    age.append(age_lst)
  else:
    age.append([])
  if medic_lst:
    medic.append(medic_lst)
  else:
    medic.append([])

In [97]:
def treatment(lst):
  treat = ""
  for i in lst:
    if i.startswith('#'):
      treat= treat+i.replace('#', '')
    else:
      treat = treat+' '+i
  return treat

final_treatment = [treatment(i) for i in medic]


In [104]:
for i,j in zip(data.transcription[:5], final_treatment[:5]):
  print(f"Transcription:  \n{i}\n\n Treatment: {j}\n")

Transcription:  
SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple with

In [72]:
age[0]

['23 - year - old', '23 - year - old']

In [57]:
get_treatment = pipeline("question-answering", model= "longluu/Medical-QA-deberta-MRQA-COVID-QA")

config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

In [ ]:
question = "What is the treatment administered to the patient?"
treatment = []
for i in data.transcription:
  treat = get_treatment(question=question, context = i)
  treatment.append(treat['answer'])

In [80]:
# question = "What is the treatment administered to the patient?"
for i in data.transcription[:10]:
  query = get_treatment(question=question, context = i)
  print(query['answer'])

 Ortho Tri-Cyclen and the Allegra.,ALLERGIES:
 gastroesophageal reflux disease.,PAST
 He is a diabetic, on medication.
 Mild mitral and tricuspid regurgitation.,2.
 mitral annular calcification extending to leaflets and supportive structures with thickening of mitral valve leaflets
 general anesthesia with endotracheal intubation.
 Dermabond, Steri-Strips and antibiotic ointment
 systemic venous return to the right atrium with normal tricuspid inflow.
 general endotracheal anesthesia,
 Ejection fraction estimated around 60%.,4.


In [61]:
get_treatment_1 = pipeline("question-answering", model ="GonzaloValdenebro/MedicalQuestionAnswering")

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [81]:
# question = "What is the treatment administered to the patient?"
for i in data.transcription[:10]:
  query = get_treatment_1(question = question, context = i)
  print(query['answer'])


Allergic rhinitis
gastroesophageal reflux disease.
meningitis,
aortic valve, mitral valve, tricuspid valve,
Doppler examination.
The patient tolerated the procedure well without any complications.
The patient was then aroused from anesthesia, extubated,
Doppler interrogation, including color flow imaging,
aspirate, which was mostly fat, little fluid,
intracardiac masses.


#### Both the models above seems to identify diseases accurately than the treatments. Have to see how it behaves when the prompt is changed.

In [82]:
question_2 = "What are the drugs administered to the patient"
for i in data.transcription[:10]:
  query = get_treatment(question=question_2, context = i)
  print(query['answer'])

 Ortho Tri-Cyclen and the Allegra.,ALLERGIES:
 Denies cellulitis, pseudotumor cerebri, meningitis, or encephalitis.,PHYSICAL
 He is a diabetic, on medication.
 Mild mitral and tricuspid regurgitation.,2.
 mitral annular calcification extending to leaflets and supportive structures with thickening of mitral valve leaflets
 general anesthesia with endotracheal intubation.
 Dermabond, Steri-Strips and antibiotic ointment
 The aortic valve is trileaflet.
 Lipodystrophy of the abdomen and thighs.,OPERATION:
 Trace mitral regurgitation.,2.  Trace tricuspid regurgitation.,IMPRESSION:,1.


In [83]:
question_2 = "What are the drugs administered to the patient"
for i in data.transcription[:10]:
  query = get_treatment_1(question=question_2, context = i)
  print(query['answer'])

loratadine
gastroesophageal reflux disease.
He smokes one and a half pack a day for 15 years,
aortic valve, mitral valve, tricuspid valve,
Doppler examination.
anvil.  I pulled the anvil into place,
The patient was then aroused from anesthesia, extubated,
Doppler interrogation, including color flow imaging,
aspirate, which was mostly fat, little fluid,
intracardiac masses.


BERT MODEL

In [105]:
get_treatment_2 = pipeline("question-answering", model= "bert-large-uncased-whole-word-masking-finetuned-squad")

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [109]:
# question = "What is the treatment administered to the patient?"
for i in data.transcription[:10]:
  query = get_treatment_2(question=question, context = i)
  print(query['answer'])

two sprays in each nostril given for three weeks
we are the appropriate program
medication
Mild mitral and tricuspid regurgitation
hyperdynamic
reverse Trendelenburg
general LMA anesthesia
Cardiac function is normal
general endotracheal anesthesia
cardiac


In [110]:
# question_2 = "What are the drugs administered to the patient"
for i in data.transcription[:10]:
  query = get_treatment_2(question=question_2, context = i)
  print(query['answer'])

Claritin, and Zyrtec
CURRENT MEDICATIONS
CURRENT MEDICATIONS:,  Include Diovan, Crestor, and Tricor
Mild mitral and tricuspid regurgitation
1.  The left ventricular cavity size and wall thickness appear normal
general anesthesia with endotracheal intubation
Dermabond, Steri-Strips and antibiotic ointment
Cardiac function is normal
general endotracheal anesthesia
cardiac chambers size.,2.  Normal left ventricular size
